In [1]:
# define the model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import optimizers
from keras.utils import to_categorical

img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

shared_fc_layer = Sequential([
    Dense(1024, activation='relu', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(), 
    Dropout(0.5),
])

encoded_a = shared_fc_layer(img_feature_a)
encoded_b = shared_fc_layer(img_feature_b)

merged_vector = concatenate([encoded_a, encoded_b])

#x = merged_vector
x = Dense(256, activation='relu')(merged_vector)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 2048)          0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 2048)          0                                            
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 256)           2365696     input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 512)           0           sequential_1[1][0]      

In [3]:
# data prepare
import cPickle as pickle
import numpy as np
import lmdb
import random

env = lmdb.open("./data/features")
txn = env.begin()

def get_XY(dataset):
    img_pairs = []
    X1 = []
    X2 = []
    Y = []
    for img_a, s_a, img_b, s_b, cmpret in dataset:
        feature_a = txn.get(img_a)
        feature_b = txn.get(img_b)
        if feature_a is None or feature_b is None:
            continue
        feature_a = np.fromstring(feature_a, np.float32)
        feature_b = np.fromstring(feature_b, np.float32)
        img_pairs.append((img_a, img_b))
        X1.append(feature_a)
        X2.append(feature_b)
        Y.append(cmpret)
    X1 = np.array(X1)
    X2 = np.array(X2)
    Y = to_categorical(np.array(Y), num_classes=2)
    return img_pairs, X1, X2, Y

batch_size = 128
train_list = pickle.load(open("./data/train.list", 'rb'))
valid_list = pickle.load(open("./data/valid.list", 'rb'))
train_img_pairs, X1_train, X2_train, Y_train = get_XY(train_list)
valid_img_pairs, X1_valid, X2_valid, Y_valid = get_XY(valid_list)

env.close()

In [4]:
# training the model
optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([X1_train, X2_train], Y_train, batch_size=batch_size, epochs=10, shuffle=True,
         validation_data=([X1_valid, X2_valid], Y_valid))

Train on 754378 samples, validate on 45499 samples
Epoch 1/10
 26368/754378 [>.............................] - ETA: 314s - loss: 0.8978 - acc: 0.5209

KeyboardInterrupt: 

In [4]:
model.save_weights("./data/rank_model.h5")

In [3]:
# 输出valid的预估结果
import cPickle as pickle

model.load_weights("./data/rank_model.h5")
Y_predict = model.predict([X1_valid, X2_valid], batch_size=batch_size)
assert(len(Y_predict) == len(valid_img_pairs))

Y_predict_map = {}
for i, (imga, imgb) in enumerate(valid_img_pairs):
    Y_predict_map[(imga, imgb)] = Y_predict[i, 1]
    
pickle.dump(Y_predict_map, open("./data/valid_predict_by_rank_model.pick", 'wb'))